In [0]:
import dlt

create stream table for orders

In [0]:
@dlt.table(
    table_properties = {"quality" : "bronze"},
    comment = "order bronze table"
)
def orders_bronze():
    df = spark.readStream.table("dev.bronze.orders_raw")
    return df

**materialized_view**

In [0]:
@dlt.table(
    table_properties = {"quality" : "bronze"},
    comment = "Customer bronze table",
    name = "customer_bronze"
)
def cust_bronze():
    df = spark.read.table("dev.bronze.customer_raw")
    return df

**dlt view**

In [0]:
@dlt.view(
    comment = "joined view"
    )
def joined_vw():
    df_c = spark.read.table("LIVE.customer_bronze")
    df_o = spark.read.table("LIVE.orders_bronze")

    df_join = df_o.join(df_c, df_o.o_custkey == df_c.c_custkey, "left_outer")

    return df_join

create MV to ADD NEW COLUMN

In [0]:
from pyspark.sql.functions import current_timestamp, count

In [0]:
@dlt.table(
    table_properties = {"quality" : "silver"},
    comment = "joined table",
    name = "joined_silver"
)
def joined_silver():
    df = spark.read.table("LIVE.joined_vw").withColumn("__insert_date", current_timestamp())
    return df

aggregate based on c_mktsegment and find the count of order

In [0]:
@dlt.table(
    table_properties={"quality": "gold"},
    comment="orders aggregated table"
)
def orders_agg_gold():
    df = spark.read.table("LIVE.joined_silver")

    df_final = df.groupBy("c_mktsegment").agg(count("o_orderkey").alias("total_orders")).withColumn("__insert_date", current_timestamp())

    return df_final